In [1]:
import gym
import gym_sokoban
import time

import matplotlib.pyplot as plt
import math
from collections import deque
import random
import numpy as np
import sys
import os
from collections import namedtuple,defaultdict,deque 

In [3]:
#import torch
#import torch.nn as nn
#import torch.nn.functional as F
#from torch.autograd import Variable
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#print(f'Using {device} device')

Using cuda:0 device


In [2]:
env_name = 'Sokoban-v1'
env = gym.make(env_name)
ACTION_LOOKUP = env.unwrapped.get_action_lookup()
print("Created environment: {}".format(env_name))


pick: xsbs/11/71.xsb, False, d:\2023-2024\RNN\Sokoban
Created environment: Sokoban-v1


In [3]:
env = gym.make(env_name)
ACTION_LOOKUP = env.unwrapped.get_action_lookup()
print("Created environment: {}".format(env_name))
env.unwrapped.set_level(0,1)
env.seed(0)
env.reset()

pick: xsbs/14/56.xsb, True, d:\2023-2024\RNN\Sokoban
Created environment: Sokoban-v1
pick: xsbs/0/1.xsb, True, d:\2023-2024\RNN\Sokoban


array([[[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0],
        [3, 0, 0, 1, 0, 0, 0, 4, 0, 0, 0, 0, 5, 3, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 

In [4]:
print('observation space:', env.observation_space.shape)
print('action space:', env.action_space)

observation space: (1, 26, 26)
action space: Discrete(8)


In [7]:
for i in env.unwrapped.room_state:
    print(i)
    for j in i:
        print (j)
    break

[3 3 3 3 3 3 3 3 3 3 3 3 3 3 0 0 0 0 0 0 0 0 0 0 0 0]
3
3
3
3
3
3
3
3
3
3
3
3
3
3
0
0
0
0
0
0
0
0
0
0
0
0


In [7]:
def value_iteration(env):
    num_iterations = 10000
    threshold = 1e-20
    gamma = 1.0
    value_table = np.zeros(env.observation_space.shape)
    for i in range(num_iterations):
        updated_value_table = np.copy(value_table)
        for state in range(env.observation_space.n):
            Q_value = [sum([p*(r+gamma*updated_value_table[s_]) for p,s_,r,_ in env.P[state][action]]) for action in range(env.action_space.n)]
            value_table[state] = max(Q_value)
        if (np.sum(np.fabs(updated_value_table - value_table)) <= threshold):
            break
    return value_table
def extract_policy(value_table):
    gamma = 1.0
    policy = np.zeros(env.observation_space.n)
    for state in range(env.observation_space.n):
        Q_table = [sum([prob*(reward + gamma*value_table[next_state]) for prob,next_state,reward,_ in env.P[state][action]]) for action in range(env.action_space.n)]
        policy[state] = np.argmax(np.array(Q_table))
    return policy
def compute_value_function(policy):
    gamma = 1.0
    value_table = np.zeros(env.observation_space.n)
    threshold = 1e-10
    while True:
        updated_value_table = np.copy(value_table)
        for state in range(env.observation_space.n):
            action = policy[state]
            value_table[state] = sum([prob*(reward + gamma*updated_value_table[next_state]) for prob,next_state,reward,_ in env.P[state][action]])
        if (np.sum((np.fabs(updated_value_table - value_table))) <= threshold):
            break
    return value_table
def policy_iteration(env):
    num_iterations = 1000
    gamma = 1.0
    policy = np.zeros(env.observation_space.n)
    for i in range(num_iterations):
        value_function = compute_value_function(policy)
        updated_policy = extract_policy(value_function)
        if (np.all(policy == updated_policy)):
            break
        policy = updated_policy
    return policy

In [ ]:
optimal_value_function = value_iteration(env=env)
optimal_policy = extract_policy(optimal_value_function)
print(optimal_policy)

In [ ]:
optimal_policy = policy_iteration(env=env)
print(optimal_policy)

### Using Policy Iteration and Value Iteration to solve

----------------

In [5]:
CHANGE_COORDINATES = {
    0: (-1, 0),
    1: (1, 0),
    2: (0, -1),
    3: (0, 1)
}
env.n = env.observation_space.shape[1] * env.observation_space.shape[2]
env.env.P = []
for x,y in range(env.n):
    #prob,next_state,reward,done in env.env.P[state][action] 
    # [(1.0, 400, -1, False)]
    for action in range(env.action_space.n):
        mapping_cord = CHANGE_COORDINATES[action % 4]
        next_state = (x + mapping_cord[0])+ ((y + mapping_cord[1]) * env.env.P.append((1.0, next_state, -1, False)))

TypeError: cannot unpack non-iterable int object

In [ ]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.env.nS)
    while True:
        # TODO: Implement!
        delta = 0  #delta = change in value of state from one iteration to next
       
        for state in range(env.env.nS):  #for all states
            val = 0  #initiate value as 0
            
            for action,act_prob in enumerate(policy[state]): #for all actions/action probabilities
                for prob,next_state,reward,done in env.env.P[state][action]:  #transition probabilities,state,rewards of each action
                    val += act_prob * prob * (reward + discount_factor * V[next_state])  #eqn to calculate
            delta = max(delta, np.abs(val-V[state]))
            V[state] = val
        if delta < theta:  #break if the change in value is less than the threshold (theta)
            break
    return np.array(V)

def policy_iteration(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """
    def one_step_lookahead(state, V):
        """
        Helper function to calculate the value for all action in a given state.
        
        Args:
            state: The state to consider (int)
            V: The value to use as an estimator, Vector of length env.nS
        
        Returns:
            A vector of length env.nA containing the expected value of each action.
        """
        A = np.zeros(env.env.nA)
        for a in range(env.env.nA):
            for prob, next_state, reward, done in env.env.P[state][a]:
                A[a] += prob * (reward + discount_factor * V[next_state])
        return A
    # Start with a random policy
    policy = np.ones([env.env.nS, env.env.nA]) / env.env.nA

    while True:
        # Implement this!
        curr_pol_val = policy_eval_fn(policy, env, discount_factor)  #eval current policy
        policy_stable = True  #Check if policy did improve (Set it as True first)
        for state in range(env.env.nS):  #for each states
            chosen_act = np.argmax(policy[state])  #best action (Highest prob) under current policy
            act_values = one_step_lookahead(state,curr_pol_val)  #use one step lookahead to find action values
            best_act = np.argmax(act_values) #find best action
            if chosen_act != best_act:
                policy_stable = False  #Greedily find best action
            policy[state] = np.eye(env.env.nA)[best_act]  #update 
        if policy_stable:
            return policy, curr_pol_val
    
    return policy, np.zeros(env.env.nS)

def value_iteration(env, theta=0.0001, discount_factor=1.0):
    """
    Value Iteration Algorithm.
    
    Args:
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
        
    Returns:
        A tuple (policy, V) of the optimal policy and the optimal value function.        
    """
    
    def one_step_lookahead(state, V):
        """
        Helper function to calculate the value for all action in a given state.
        
        Args:
            state: The state to consider (int)
            V: The value to use as an estimator, Vector of length env.nS
        
        Returns:
            A vector of length env.nA containing the expected value of each action.
        """
        A = np.zeros(env.env.nA)
        for act in range(env.env.nA):
            for prob, next_state, reward, done in env.env.P[state][act]:
                A[act] += prob * (reward + discount_factor*V[next_state])
        return A

In [9]:
env = gym.make('Sokoban-v1')
env.reset()
for i_episode in range(1):#20
    observation = env.reset()
    steps = 1000
    last_update = time.time()
    while steps > 0:
        env.render(mode='human')
        if time.time() - last_update < .2:
            continue
        last_update = time.time()
        steps -= 1
        action = env.action_space.sample()
        # Sleep makes the actions visible for users
        observation, reward, done, info = env.step(action)

        print(ACTION_LOOKUP[action], reward, done, info)
        if done:
            print("Episode finished")
            env.render()
            break

    env.close()

time.sleep(10)

pick: xsbs/0/1.xsb, True, d:\2023-2024\RNN\Sokoban
pick: xsbs/0/1.xsb, True, d:\2023-2024\RNN\Sokoban
pick: xsbs/0/1.xsb, True, d:\2023-2024\RNN\Sokoban
move up 1.9 False {'action.name': 'move up', 'action.moved_player': True, 'action.moved_box': False}
push up 1.7999999999999998 False {'action.name': 'push up', 'action.moved_player': True, 'action.moved_box': False}
move left 1.6999999999999997 False {'action.name': 'move left', 'action.moved_player': False, 'action.moved_box': False}
push up 1.5999999999999996 False {'action.name': 'push up', 'action.moved_player': True, 'action.moved_box': False}
push right 1.4999999999999996 False {'action.name': 'push right', 'action.moved_player': True, 'action.moved_box': False}
push down 1.3999999999999995 False {'action.name': 'push down', 'action.moved_player': False, 'action.moved_box': False}
push right 1.2999999999999994 False {'action.name': 'push right', 'action.moved_player': True, 'action.moved_box': False}
move up 1.1999999999999993 F

KeyboardInterrupt: 

## Implement MDP and temporal difference learning
-------------------

In [7]:
def epsilon_greedy(Q,state, nA,eps):
    if np.random.random() > eps:
        return np.argmax(Q[state])
    else:
        return np.random.choice(np.arange(env.action_space.n))
    
def update_Q_sarsamax(alpha, gamma, Q, state, action, reward, next_state=None):
    current = Q[state][action]         # estimate in Q-table (for current state, action pair)
    Qsa_next = np.max(Q[next_state]) if next_state is not None else 0  # value of next state 
    target = reward + (gamma * Qsa_next)               # construct TD target
    new_value = current + (alpha * (target - current)) # get updated value 
    return new_value

In [ ]:
def sarsa(env,num_eps, alpha, gamma=1.0,plot_interval=200):
    nA = env.action_space.n
    Q = defaultdict(lambda: np.zeros(nA))

    tmp_scores = deque(maxlen=plot_interval)
    avg_scores = deque(maxlen=num_eps)
    for i_eps in range(1,num_eps+1):
        if i_eps % plot_interval == 0:
            print("\rEpisode {}/{}".format(i_eps, num_eps), end="")
            sys.stdout.flush()
        score = 0
        state = env.reset()
        eps = 1.0 / i_eps
        action = epsilon_greedy(Q,state, nA,eps)
        while True:
            next_state, reward, done, info = env.step(action)
            score += reward
            if not done:
                next_action = epsilon_greedy(Q,next_state, nA,eps)
                Q[state][action] = update_Q_sarsamax(alpha, gamma, Q, \
                                                     state, action, reward, next_state)
                state = next_state
                action = next_action
            if done:
                Q[state][action] = update_Q_sarsamax(alpha, gamma, Q, \
                                                     state, action, reward)
                tmp_scores.append(score)
                break
        if (i_eps % plot_interval == 0):
            avg_scores.append(np.mean(tmp_scores))
    plt.plot(np.linspace(0,num_eps,len(avg_scores),endpoint=False),np.asarray(avg_scores))
    plt.xlabel('Episode Number')
    plt.ylabel('Average Reward (Over Next %d Episodes)' % plot_interval)
    plt.show()
    print(('Best Average Reward over %d Episodes: ' % plot_interval), np.max(avg_scores))
    return Q


## Monte Carlo menthod
-------------------

In [6]:
env = gym.make(env_name)
ACTION_LOOKUP = env.unwrapped.get_action_lookup()
print("Created environment: {}".format(env_name))
env.unwrapped.set_level(0,1)
env.seed(0)
np.random.seed(0)
_ = env.reset()

pick: xsbs/0/1.xsb, True, d:\2023-2024\RNN\Sokoban
Created environment: Sokoban-v1
pick: xsbs/0/1.xsb, True, d:\2023-2024\RNN\Sokoban


In [4]:
def greedy_policy(V,x,y):
    #.9 prob of greedy action
    #.1 prob of random action
    r_choice = np.choose([0,1],[.1,.9])
    if np.random.random() < r_choice:
        return np.random.choice(np.arange(env.action_space.n))
    else:
        return np.argmax(V[x,y])
def find_player(observation):
    for x in range(observation.shape[1]):
        for y in range(observation.shape[2]):
            if observation[0,x,y] == 1:
                return x,y
    return -1,-1

In [21]:
episodes = 200
gamma = .9
obs = (env.observation_space.shape[1],env.observation_space.shape[2],len(ACTION_LOOKUP))
V = np.zeros(obs) + 1
Returns = defaultdict(list)

for episode in range(episodes):
    env = gym.make(env_name)
    _ = env.reset()
    x,y = find_player(_)
    for i in range(1000):
        action = greedy_policy(V,x,y)
        observation, reward, done, info = env.step(action)
        V[x,y,action] = gamma*V[x,y,action] + reward



        x,y = find_player(observation)
        print(ACTION_LOOKUP[action], reward, done, info)
        if done:
            print("Episode finished")
            env.render()
            break


In [22]:
V.shape

(26, 26)